In [2]:
import os
import shutil
import scipy.io
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#paths
labels_path = "/content/drive/MyDrive/oxford102/imagelabels.mat"
img_path = "/content/drive/MyDrive/oxford102/flowers"
output_dir = "/content/drive/MyDrive/oxford102/flower_dataset"

In [ ]:
# Load labels
labels = scipy.io.loadmat(labels_path)["labels"][0]  # shape: (8189,)
num_images = len(labels)  # should be 8189
print(f"Loaded {num_images} labels")

Loaded 8189 labels


In [ ]:
from tqdm import tqdm

In [ ]:
# Create class folders
# ----------------------
for class_id in range(1, 103):  # 102 classes, 1-indexed
    class_folder = os.path.join(output_dir, str(class_id))
    os.makedirs(class_folder, exist_ok=True)

# ----------------------
# Copy images into folders
# ----------------------
for i, label in tqdm(enumerate(labels, start=1), total=num_images):
    img_name = f"image_{i:05d}.jpg"  # adjust if needed (04d or no padding)
    src = os.path.join(img_path, img_name)
    dst = os.path.join(output_dir, str(label), img_name)

    if os.path.exists(src):
        os.makedirs(os.path.dirname(dst), exist_ok=True)  # <-- ensures class folder exists
        shutil.copyfile(src, dst)
    else:
        print(f"Missing file: {src}")

print("✅ Dataset organized for PyTorch ImageFolder!")

100%|██████████| 8189/8189 [40:21<00:00,  3.38it/s]

✅ Dataset organized for PyTorch ImageFolder!


In [ ]:
import numpy as np

In [ ]:
# Data Split

labels_path = "/content/drive/MyDrive/oxford102/imagelabels.mat"
img_path = "/content/drive/MyDrive/oxford102/flowers"
output_dir = "/content/drive/MyDrive/oxford102/flower_train_test"

In [ ]:
labels = scipy.io.loadmat(labels_path)["labels"][0]  # shape: (8189,)
num_images = len(labels)
print(f"Loaded {num_images} labels")

Loaded 8189 labels


In [ ]:
split_ratios = {"train": 0.7, "val": 0.15, "test": 0.15}

# ----------------------
# Organize dataset
# ----------------------
for class_id in range(1, 103):  # 102 classes (1-indexed)
    # find indices of images belonging to this class
    indices = [i for i, lbl in enumerate(labels, start=1) if lbl == class_id]
    np.random.shuffle(indices)

    # split indices
    n_total = len(indices)
    n_train = int(n_total * split_ratios["train"])
    n_val = int(n_total * split_ratios["val"])

    train_idx = indices[:n_train]
    val_idx = indices[n_train:n_train + n_val]
    test_idx = indices[n_train + n_val:]

    splits = {"train": train_idx, "val": val_idx, "test": test_idx}

    for split_name, split_indices in splits.items():
        for i in split_indices:
            img_name = f"image_{i:05d}.jpg"  # adjust if your dataset uses 04d or no padding
            src = os.path.join(img_path, img_name)
            dst = os.path.join(output_dir, split_name, str(class_id), img_name)

            if os.path.exists(src):
                os.makedirs(os.path.dirname(dst), exist_ok=True)
                shutil.copyfile(src, dst)
            else:
                print(f"Missing file: {src}")

print("✅ Dataset organized into train/val/test for ImageFolder!")

✅ Dataset organized into train/val/test for ImageFolder!


# Start training the Model

In [3]:
import torch
import torch.nn as nn
import torchvision.models as models

In [4]:
# Model: ResNet50

model = models.resnet50(pretrained=True)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 219MB/s]


In [5]:
num_features = model.fc.in_features

In [6]:
model.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512,102)
)


checkpoint_path = "/content/drive/MyDrive/oxford102/checkpoint_epoch5.pth"
model.load_state_dict(torch.load(checkpoint_path))

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [8]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [9]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(),
    transforms.ToTensor(),
])

In [20]:

train_dataset = datasets.ImageFolder("/content/drive/MyDrive/oxford102/flower_train_test/train", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = datasets.ImageFolder("/content/drive/MyDrive/oxford102/flower_train_test/val", transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32)

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")




def train(model,epochs):
  model.to(device)

  for epoch in range(epochs):
    model.train()
    running_loss=0.0
    for b, (X,y) in enumerate(train_loader):
      X, y = X.to(device), y.to(device)

      optimizer.zero_grad()
      y_pred = model(X)
      loss = criterion(y_pred,y)

      loss.backward()
      optimizer.step()

      running_loss += loss.item()

      if (b + 1) % 10 == 0:  # every 10 batches
        print(f"Epoch [{epoch+1}/{epochs}], Batch [{b+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

    torch.save(model.state_dict(), f"/content/drive/MyDrive/oxford102/checkpoint_epoch{epoch+1}.pth")

In [14]:
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True



In [20]:
train(model,5)

Epoch [1/5], Batch [10/178], Loss: 4.4000
Epoch [1/5], Batch [20/178], Loss: 4.1407
Epoch [1/5], Batch [30/178], Loss: 4.3769
Epoch [1/5], Batch [40/178], Loss: 4.0502
Epoch [1/5], Batch [50/178], Loss: 4.1808
Epoch [1/5], Batch [60/178], Loss: 3.6928
Epoch [1/5], Batch [70/178], Loss: 4.0439
Epoch [1/5], Batch [80/178], Loss: 3.9556
Epoch [1/5], Batch [90/178], Loss: 3.9628
Epoch [1/5], Batch [100/178], Loss: 4.0112
Epoch [1/5], Batch [110/178], Loss: 3.8555
Epoch [1/5], Batch [120/178], Loss: 3.7279
Epoch [1/5], Batch [130/178], Loss: 3.7231
Epoch [1/5], Batch [140/178], Loss: 3.7866
Epoch [1/5], Batch [150/178], Loss: 3.5763
Epoch [1/5], Batch [160/178], Loss: 3.5839
Epoch [1/5], Batch [170/178], Loss: 3.6988
Epoch 1/5, Loss: 3.9453
Epoch [2/5], Batch [10/178], Loss: 3.1292
Epoch [2/5], Batch [20/178], Loss: 3.2734
Epoch [2/5], Batch [30/178], Loss: 3.2251
Epoch [2/5], Batch [40/178], Loss: 3.1153
Epoch [2/5], Batch [50/178], Loss: 3.3382
Epoch [2/5], Batch [60/178], Loss: 2.9704
Ep

In [21]:
torch.save(model.state_dict(), "/content/drive/MyDrive/oxford102/resnet50_flowers.pth")

In [16]:
model.to(device)
model.train()
X, y = next(iter(train_loader))
X, y = X.to(device), y.to(device)
y_pred = model(X)
loss = criterion(y_pred, y)
print("Batch processed, loss:", loss.item())


Batch processed, loss: 4.563467025756836


#Load in and test model accuracy

In [5]:
import torch
import torch.nn as nn
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet50(weights="IMAGENET1K_V1")
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 102)
)

# Load trained weights
model.load_state_dict(torch.load("/content/drive/MyDrive/oxford102/resnet50_flowers.pth"))
model.to(device)

# model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [8]:
import pandas as pd

flower_names = pd.read_csv('/content/drive/MyDrive/oxford102/oxford_flower_102_name.csv')
flower_names.head()

,Index,Name
0,0,pink primrose
1,1,hard-leaved pocket orchid
2,2,canterbury bells
3,3,sweet pea
4,4,english marigold


In [12]:
flowers = list(flower_names['Name'].unique())

In [13]:
flowers

['pink primrose',
 'hard-leaved pocket orchid',
 'canterbury bells',
 'sweet pea',
 'english marigold',
 'tiger lily',
 'moon orchid',
 'bird of paradise',
 'monkshood',
 'globe thistle',
 'snapdragon',
 "colt's foot",
 'king protea',
 'spear thistle',
 'yellow iris',
 'globe-flower',
 'purple coneflower',
 'peruvian lily',
 'balloon flower',
 'giant white arum lily',
 'fire lily',
 'pincushion flower',
 'fritillary',
 'red ginger',
 'grape hyacinth',
 'corn poppy',
 'prince of wales feathers',
 'stemless gentian',
 'artichoke',
 'sweet william',
 'carnation',
 'garden phlox',
 'love in the mist',
 'mexican aster',
 'alpine sea holly',
 'ruby-lipped cattleya',
 'cape flower',
 'great masterwort',
 'siam tulip',
 'lenten rose',
 'barbeton daisy',
 'daffodil',
 'sword lily',
 'poinsettia',
 'bolero deep blue',
 'wallflower',
 'marigold',
 'buttercup',
 'oxeye daisy',
 'common dandelion',
 'petunia',
 'wild pansy',
 'primula',
 'sunflower',
 'pelargonium',
 'bishop of llandaff',
 'gaura',

In [14]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [15]:
# get the test data

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(),
    transforms.ToTensor(),
])

In [30]:
def evaluate(model, dataloader, criterion, device):
    model.eval()  # set model to evaluation mode
    running_loss = 0.0
    correct = 0
    total = 0
    max_batches = 20

    with torch.no_grad():
      for i, (X, y) in enumerate(dataloader):
        if i >= max_batches:
          break  # stop after first `max_batches` batches
        X,y = X.to(device), y.to(device)

        outputs = model(X)
        loss = criterion(outputs,y)
        running_loss += loss.item()

        #predictions
        _, preds = torch.max(outputs,1)
        correct += (preds == y).sum().item()
        total += y.size(0)

    avg_loss = running_loss / len(dataloader)
    accuracy = 100 * correct / total
    print(f"Validation Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
    return avg_loss, accuracy

In [31]:
criterion = nn.CrossEntropyLoss()


evaluate(model, val_loader,criterion,device)

Validation Loss: 0.7090, Accuracy: 76.25%


(0.7089956065541819, 76.25)

#Testing predictions


In [32]:
def predict(model, X, device, class_names):
    model.eval()
    with torch.no_grad():
        X = X.to(device)
        outputs = model(X)
        _, preds = torch.max(outputs, 1)
    return [class_names[p] for p in preds.cpu().numpy()]

In [35]:
test_data = datasets.ImageFolder("/content/drive/MyDrive/oxford102/flower_train_test/test", transform=transform)
test_loader = DataLoader(test_data, batch_size=10, shuffle=True)

In [40]:
import random

idx = random.randint(0, len(test_data)-1)
X, y = test_data[idx]
print(idx)
print(X)
print(y)

269
tensor([[[0.1333, 0.1647, 0.2000,  ..., 0.0353, 0.0431, 0.0471],
         [0.1176, 0.1490, 0.1882,  ..., 0.0353, 0.0431, 0.0471],
         [0.1137, 0.1412, 0.1804,  ..., 0.0314, 0.0392, 0.0431],
         ...,
         [0.3961, 0.3922, 0.4039,  ..., 0.1647, 0.1608, 0.1882],
         [0.4196, 0.4118, 0.4000,  ..., 0.1686, 0.1765, 0.2000],
         [0.4157, 0.4118, 0.4039,  ..., 0.1882, 0.1961, 0.2078]],

        [[0.2353, 0.2745, 0.3255,  ..., 0.0392, 0.0471, 0.0510],
         [0.2118, 0.2510, 0.3059,  ..., 0.0392, 0.0471, 0.0510],
         [0.1882, 0.2275, 0.2824,  ..., 0.0353, 0.0431, 0.0471],
         ...,
         [0.5686, 0.5647, 0.5765,  ..., 0.3255, 0.3255, 0.3569],
         [0.5765, 0.5725, 0.5647,  ..., 0.3294, 0.3412, 0.3686],
         [0.5686, 0.5686, 0.5647,  ..., 0.3451, 0.3569, 0.3765]],

        [[0.0902, 0.1216, 0.1647,  ..., 0.0196, 0.0275, 0.0314],
         [0.0706, 0.1020, 0.1490,  ..., 0.0196, 0.0275, 0.0314],
         [0.0510, 0.0824, 0.1294,  ..., 0.0157, 0.0235

(tensor([[[0.1333, 0.1647, 0.2000,  ..., 0.0353, 0.0431, 0.0471],
          [0.1176, 0.1490, 0.1882,  ..., 0.0353, 0.0431, 0.0471],
          [0.1137, 0.1412, 0.1804,  ..., 0.0314, 0.0392, 0.0431],
          ...,
          [0.3961, 0.3922, 0.4039,  ..., 0.1647, 0.1608, 0.1882],
          [0.4196, 0.4118, 0.4000,  ..., 0.1686, 0.1765, 0.2000],
          [0.4157, 0.4118, 0.4039,  ..., 0.1882, 0.1961, 0.2078]],
 
         [[0.2353, 0.2745, 0.3255,  ..., 0.0392, 0.0471, 0.0510],
          [0.2118, 0.2510, 0.3059,  ..., 0.0392, 0.0471, 0.0510],
          [0.1882, 0.2275, 0.2824,  ..., 0.0353, 0.0431, 0.0471],
          ...,
          [0.5686, 0.5647, 0.5765,  ..., 0.3255, 0.3255, 0.3569],
          [0.5765, 0.5725, 0.5647,  ..., 0.3294, 0.3412, 0.3686],
          [0.5686, 0.5686, 0.5647,  ..., 0.3451, 0.3569, 0.3765]],
 
         [[0.0902, 0.1216, 0.1647,  ..., 0.0196, 0.0275, 0.0314],
          [0.0706, 0.1020, 0.1490,  ..., 0.0196, 0.0275, 0.0314],
          [0.0510, 0.0824, 0.1294,  ...,

# With the code below we can use the test data to test unseen images.

In [50]:
img, label = test_data[205]

# add batch dimension
img = img.unsqueeze(0)  # shape: [1, C, H, W]

# predict
pred_name = predict(model, img, device, flowers)[0]

print(f"Predicted: {pred_name}, Actual: {flowers[label]}")

Predicted: fritillary, Actual: fritillary
